## Multivariate models(ucm,xgboost,prophet,Lstm) with moving average concept for future prediction

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_excel(r"C:\Users\HarshaGupta\Downloads\forecasting_tool_feature_engineered_sales_data 1.xlsx")

In [3]:
df.isnull().sum()

date                     0
product_id               0
cpi                      0
fuel_price               0
unemployment             0
gdp_growth               0
consumer_confidence      0
interest_rate            0
social_media             0
non_social_media         0
ad_spend                 0
retail_traffic           0
competitor_ad_spend      0
product_price            0
promo_discount           0
inventory_level          0
holiday_indicator        0
monthly_sales            0
sales_lag1              15
ad_spend_lag1           15
social_media_lag1       15
social_media_roll3       0
retail_traffic_roll3     0
dtype: int64

In [4]:
#replace all null values with sum of first 3 non-null from original column

In [5]:

lag_info = {
    'sales_lag1': 'monthly_sales',
    'ad_spend_lag1': 'ad_spend',
    'social_media_lag1': 'social_media'
}

# Ensure sorting by product_id and date
df = df.sort_values(['product_id', 'date'])

def fill_lag_nans(group, lag_col, original_col):
    # Take first 3 non-null values from the original column
    first_3_values = group[original_col].dropna().iloc[:3]
    mean_first_3 = first_3_values.mean()
    # Fill NaNs in lag column with this mean
    group[lag_col] = group[lag_col].fillna(mean_first_3)
    return group

# Apply for each lag column
for lag_col, original_col in lag_info.items():
    df = df.groupby('product_id').apply(lambda group: fill_lag_nans(group, lag_col, original_col)).reset_index(drop=True)


C:\Users\HarshaGupta\AppData\Local\Temp\ipykernel_3092\393433063.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('product_id').apply(lambda group: fill_lag_nans(group, lag_col, original_col)).reset_index(drop=True)
C:\Users\HarshaGupta\AppData\Local\Temp\ipykernel_3092\393433063.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('product_id').apply(lambda group: fill_l

In [6]:
df.isnull().sum()

date                    0
product_id              0
cpi                     0
fuel_price              0
unemployment            0
gdp_growth              0
consumer_confidence     0
interest_rate           0
social_media            0
non_social_media        0
ad_spend                0
retail_traffic          0
competitor_ad_spend     0
product_price           0
promo_discount          0
inventory_level         0
holiday_indicator       0
monthly_sales           0
sales_lag1              0
ad_spend_lag1           0
social_media_lag1       0
social_media_roll3      0
retail_traffic_roll3    0
dtype: int64

## ucm 

In [9]:
# 2 months

In [10]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_percentage_error, r2_score
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

feature_cols = ['product_price', 'inventory_level', 'ad_spend_lag1', 'social_media_roll3', 'social_media_lag1',
                'ad_spend', 'non_social_media', 'social_media', 'competitor_ad_spend', 'retail_traffic_roll3',
                'cpi', 'unemployment', 'fuel_price', 'consumer_confidence', 'gdp_growth', 'interest_rate',
                'promo_discount', 'retail_traffic', 'holiday_indicator']
target_col = 'monthly_sales'
look_back = 3

def calculate_mape(true_values, predicted_values):
    true_values = np.array(true_values)
    predicted_values = np.array(predicted_values)
    mask = true_values != 0
    return np.mean(np.abs((true_values[mask] - predicted_values[mask]) / true_values[mask])) * 100

final_res_df = pd.DataFrame()
train_fitted_list = []

for item_id in df['product_id'].unique():
    try:
        id_data = df[df['product_id'] == item_id].copy()
        id_data = id_data.sort_values('date')
        id_data.reset_index(drop=True, inplace=True)
        id_data['date'] = pd.to_datetime(id_data['date'])

        # Updated train/test split here:
        train_data = id_data[id_data['date'] < '2024-11-01'].copy()
        test_data = id_data[(id_data['date'] >= '2024-11-01') & (id_data['date'] <= '2024-12-01')].copy()

        if len(train_data) < look_back + 1 or test_data.empty:
            continue

        model = sm.tsa.UnobservedComponents(
            train_data[target_col],
            level='local level',
            trend=True,
            exog=train_data[feature_cols]
        )
        model_fitted = model.fit(disp=False)

        train_fitted = model_fitted.fittedvalues
        y_train_actual = train_data[target_col].iloc[look_back:]
        y_train_pred = train_fitted.iloc[look_back:]
        mape_train = calculate_mape(y_train_actual, y_train_pred)
        r2_train = r2_score(y_train_actual, y_train_pred)

        test_exog = test_data[feature_cols]
        forecast_test = model_fitted.get_forecast(steps=len(test_data), exog=test_exog)
        test_forecast_values = forecast_test.predicted_mean.tolist()
        test_actual = test_data[target_col].values
        mape_test = calculate_mape(test_actual, test_forecast_values)
        r2_test = r2_score(test_actual, test_forecast_values)

        future_months = pd.to_datetime(['2025-01-01', '2025-02-01'])
        predicted_sales = []
        extended_df = id_data.copy()

        for i, pred_month in enumerate(future_months):
            if i == 0:
                # For Jan 2025, use Nov 2024 and Dec 2024
                ma_window = pd.to_datetime(['2024-11-01', '2024-12-01'])
            elif i == 1:
                # For Feb 2025, use Dec 2024 and Jan 2025
                ma_window = pd.to_datetime(['2024-12-01', '2025-01-01'])

            ma_rows = []
            for win_month in ma_window:
                if win_month in extended_df['date'].values:
                    row = extended_df[extended_df['date'] == win_month].iloc[0]
                else:
                    # If predicted month not in data, get from predicted_sales
                    idx = list(future_months).index(win_month)
                    predicted_val = predicted_sales[idx]
                    prev_month = future_months[idx - 1] if idx > 0 else ma_window[0]
                    prev_features_row = extended_df[extended_df['date'] == prev_month].iloc[0]

                    row_dict = {'date': win_month, target_col: predicted_val}
                    for col in feature_cols:
                        row_dict[col] = prev_features_row[col]
                    row = pd.Series(row_dict)
                ma_rows.append(row)

            ma_data = pd.DataFrame(ma_rows)

            # Compute moving average of features
            exog_ma = ma_data[feature_cols].mean().to_frame().T
            forecast_future = model_fitted.get_forecast(steps=1, exog=exog_ma)
            next_pred = forecast_future.predicted_mean.iloc[0]
            predicted_sales.append(next_pred)

            # Append this prediction to extended_df for future iterations
            new_row = {'date': pred_month, target_col: next_pred}
            for col in feature_cols:
                new_row[col] = exog_ma[col].values[0]
            new_row['product_id'] = item_id
            extended_df = pd.concat([extended_df, pd.DataFrame([new_row])], ignore_index=True)

        forecast_months = ['2024-11-01', '2024-12-01', '2025-01-01', '2025-02-01']
        actuals = test_actual.tolist() + [np.nan] * 2
        predictions = test_forecast_values + predicted_sales
        mape_values = [mape_test] * len(test_forecast_values) + [np.nan] * 2
        r2_values = [r2_test] * len(test_forecast_values) + [np.nan] * 2

        store_res_df = pd.DataFrame({
            'id': [item_id] * len(forecast_months),
            'month': forecast_months,
            'actual': actuals,
            'predicted': predictions,
            'train_mape': [mape_train] * len(forecast_months),
            'train_r2': [r2_train] * len(forecast_months),
            'test_mape': mape_values,
            'test_r2': r2_values
        })

        final_res_df = pd.concat([final_res_df, store_res_df], axis=0)

        train_months = train_data['date'].iloc[look_back:].values
        train_fitted_df = pd.DataFrame({
            'id': [item_id] * len(train_months),
            'month': train_months,
            'actual': y_train_actual.values,
            'train_fitted': y_train_pred.values,
            'train_mape': [mape_train] * len(train_months),
            'train_r2': [r2_train] * len(train_months)
        })
        train_fitted_list.append(train_fitted_df)

    except Exception as e:
        print(f"Model failed for ID {item_id}: {e}")
        continue

final_res_df.reset_index(drop=True, inplace=True)
all_train_fitted_df = pd.concat(train_fitted_list, axis=0).reset_index(drop=True)

print(final_res_df)
print(all_train_fitted_df)


      id       month     actual     predicted  train_mape  train_r2  \
0   P001  2024-11-01   447383.0  4.691776e+05    1.799214  0.890281   
1   P001  2024-12-01   444480.0  4.625493e+05    1.799214  0.890281   
2   P001  2025-01-01        NaN  4.658635e+05    1.799214  0.890281   
3   P001  2025-02-01        NaN  4.642064e+05    1.799214  0.890281   
4   P002  2024-11-01   925066.0  8.969183e+05    1.455656  0.927033   
5   P002  2024-12-01   919084.0  9.096235e+05    1.455656  0.927033   
6   P002  2025-01-01        NaN  9.032709e+05    1.455656  0.927033   
7   P002  2025-02-01        NaN  9.064472e+05    1.455656  0.927033   
8   P003  2024-11-01   813813.0  8.425776e+05    1.495501  0.916553   
9   P003  2024-12-01   731467.0  7.602164e+05    1.495501  0.916553   
10  P003  2025-01-01        NaN  8.013970e+05    1.495501  0.916553   
11  P003  2025-02-01        NaN  7.808067e+05    1.495501  0.916553   
12  P004  2024-11-01   461406.0  4.576430e+05    1.584146  0.916742   
13  P0

In [11]:
# all_train_fitted_df.to_excel('ucm_final_2_MA.xlsx',index=False)
# final_res_df.to_excel('ucm_final_2_forecasting_MA.xlsx',index=False)

In [7]:
#3 months

In [8]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_percentage_error, r2_score
import warnings

# Suppress convergence warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=Rfinal_res_df.to_excel('xgboost_final_12_forecasting_MA.xlsx',index=False)untimeWarning)

# Feature and target columns
feature_cols = ['product_price', 'inventory_level', 'ad_spend_lag1', 'social_media_roll3', 'social_media_lag1',
                'ad_spend', 'non_social_media', 'social_media', 'competitor_ad_spend', 'retail_traffic_roll3',
                'cpi', 'unemployment', 'fuel_price', 'consumer_confidence', 'gdp_growth', 'interest_rate',
                'promo_discount', 'retail_traffic', 'holiday_indicator']
target_col = 'monthly_sales'
look_back = 3

# Function to calculate MAPE
def calculate_mape(true_values, predicted_values):
    true_values = np.array(true_values)
    predicted_values = np.array(predicted_values)
    mask = true_values != 0
    return np.mean(np.abs((true_values[mask] - predicted_values[mask]) / true_values[mask])) * 100

# Result placeholders
final_res_df = pd.DataFrame()
train_fitted_list = []

# Loop through each product_id
for item_id in df['product_id'].unique():
    try:
        id_data = df[df['product_id'] == item_id].copy()
        id_data = id_data.sort_values('date')
        id_data.reset_index(drop=True, inplace=True)

        # Ensure date is in datetime format
        id_data['date'] = pd.to_datetime(id_data['date'])

        train_data = id_data[id_data['date'] < '2024-10-01'].copy()
        test_data = id_data[(id_data['date'] >= '2024-10-01') & (id_data['date'] <= '2024-12-01')].copy()

        if len(train_data) < look_back + 1 or test_data.empty:
            continue  # Skip if not enough data

        # Fit UCM model on training data
        model = sm.tsa.UnobservedComponents(
            train_data[target_col],
            level='local level',
            trend=True,
            exog=train_data[feature_cols]
        )
        model_fitted = model.fit(disp=False)

        # Training accuracy
        train_fitted = model_fitted.fittedvalues
        y_train_actual = train_data[target_col].iloc[look_back:]
        y_train_pred = train_fitted.iloc[look_back:]
        mape_train = calculate_mape(y_train_actual, y_train_pred)
        r2_train = r2_score(y_train_actual, y_train_pred)

        # Forecast for Oct–Dec 2024
        test_exog = test_data[feature_cols]
        forecast_test = model_fitted.get_forecast(steps=3, exog=test_exog)
        test_forecast_values = forecast_test.predicted_mean.tolist()
        test_actual = test_data[target_col].values
        mape_test = calculate_mape(test_actual, test_forecast_values)
        r2_test = r2_score(test_actual, test_forecast_values)

        # Predict Jan–Mar 2025 using moving average of exog
        future_months = pd.to_datetime(['2025-01-01', '2025-02-01', '2025-03-01'])
        predicted_sales = []

        # Create a copy of original data to extend with predicted values
        extended_df = id_data.copy()

        for i, pred_month in enumerate(future_months):
            if i == 0:
                ma_window = pd.to_datetime(['2024-10-01', '2024-11-01', '2024-12-01'])
            elif i == 1:
                ma_window = pd.to_datetime(['2024-11-01', '2024-12-01', '2025-01-01'])
            else:
                ma_window = pd.to_datetime(['2024-12-01', '2025-01-01', '2025-02-01'])

            # Initialize empty list to collect rows
            ma_rows = []

            for win_month in ma_window:
                if win_month in extended_df['date'].values:
                    row = extended_df[extended_df['date'] == win_month].iloc[0]
                else:
                    idx = list(future_months).index(win_month)
                    predicted_val = predicted_sales[idx]
                    prev_month = future_months[idx - 1] if idx > 0 else ma_window[0]
                    prev_features_row = extended_df[extended_df['date'] == prev_month].iloc[0]

                    row_dict = {'date': win_month, target_col: predicted_val}
                    for col in feature_cols:
                        row_dict[col] = prev_features_row[col]
                    row = pd.Series(row_dict)
                ma_rows.append(row)

            ma_data = pd.DataFrame(ma_rows)
            # ⬇️ Print the moving average input data
            print(f"\nMoving average window for {pred_month}: {ma_window}")
            print(ma_data[['date'] + feature_cols])

            # Compute moving average of features
            exog_ma = ma_data[feature_cols].mean().to_frame().T

    #         # # ⬇️ Print the calculated moving average of exogenous variables
    #         # print(f"\nCalculated exog MA for {pred_month}:")
    #         # print(exog_ma)

    #         forecast = model_fitted.get_forecast(steps=1, exog=exog_ma)
    #         pred_val = forecast.predicted_mean.iloc[0]
    #         predicted_sales.append(pred_val)

    #         new_row = {'product_id': item_id, 'date': pred_month, target_col: pred_val}
    #         for col in feature_cols:
    #             new_row[col] = exog_ma[col].values[0]
    #         extended_df = pd.concat([extended_df, pd.DataFrame([new_row])], ignore_index=True)

    # except Exception as e:
    #     print(f"Error for product_id {item_id}: {e}")
    #     continue

#             exog_ma = ma_data[feature_cols].mean().to_frame().T

            forecast_future = model_fitted.get_forecast(steps=1, exog=exog_ma)
            next_pred = forecast_future.predicted_mean.iloc[0]
            predicted_sales.append(next_pred)

            # Append this new prediction to extended_df
            new_row = {'date': pred_month, target_col: next_pred}
            for col in feature_cols:
                new_row[col] = exog_ma[col].values[0]
            new_row['product_id'] = item_id
            extended_df = pd.concat([extended_df, pd.DataFrame([new_row])], ignore_index=True)
        
        # Store results
        forecast_months = ['2024-10-01', '2024-11-01', '2024-12-01',
                           '2025-01-01', '2025-02-01', '2025-03-01']
        actuals = test_actual.tolist() + [np.nan] * 3
        predictions = test_forecast_values + predicted_sales
        mape_values = [mape_test] * 3 + [np.nan] * 3
        r2_values = [r2_test] * 3 + [np.nan] * 3

        store_res_df = pd.DataFrame({
            'id': [item_id] * 6,
            'month': forecast_months,
            'actual': actuals,
            'predicted': predictions,
            'train_mape': [mape_train] * 6,
            'train_r2': [r2_train] * 6,
            'test_mape': mape_values,
            'test_r2': r2_values
        })

        final_res_df = pd.concat([final_res_df, store_res_df], axis=0)

        # Store train fitted values
        train_months = train_data['date'].iloc[look_back:].values

        train_fitted_df = pd.DataFrame({
            'all_train_fitted_df.to_excel('xgboost_final_12_MA.xlsx',index=False)id': [item_id] * len(train_months),
            'month': train_months,
            'actual': y_train_actual.values,
            'train_fitted': y_train_pred.values,
            'train_mape': [mape_train] * len(train_months),
            'train_r2': [r2_train] * len(train_months)
        })

        train_fitted_list.append(train_fitted_df)

    except Exception as e:
        print(f"Model failed for ID {item_id}: {e}")
        continue

# Final result consolidation
final_res_df.reset_index(drop=True, inplace=True)
all_train_fitted_df = pd.concat(train_fitted_list, axis=0).reset_index(drop=True)

# Show results
print(final_res_df)
print(all_train_fitted_df)




Moving average window for 2025-01-01 00:00:00: DatetimeIndex(['2024-10-01', '2024-11-01', '2024-12-01'], dtype='datetime64[ns]', freq=None)
          date  product_price  inventory_level  ad_spend_lag1  \
117 2024-10-01          97.61             4343         4755.0   
118 2024-11-01         101.21             4420         6388.0   
119 2024-12-01         104.12             4269         6983.0   

     social_media_roll3  social_media_lag1  ad_spend  non_social_media  \
117         1789.000000             1480.0      6388              2279   
118         1817.666667             2113.0      6983              2546   
119         1999.666667             1860.0      6048              2231   

     social_media  competitor_ad_spend  retail_traffic_roll3   cpi  \
117          2113                 5043           5830.333333  2.78   
118          1860                 8324           5743.333333  2.72   
119          2026                 6446           5824.333333  2.66   

     unemployment  f

In [ ]:
# all_train_fitted_df.to_excel('ucm_final_3_MA.xlsx',index=False)
# final_res_df.to_excel('ucm_final_3_forecasting_MA.xlsx',index=False)

In [12]:
# 1 year 

In [13]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_percentage_error, r2_score
import warnings

# Suppress convergence warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Feature and target columns
feature_cols = ['product_price', 'inventory_level', 'ad_spend_lag1', 'social_media_roll3', 'social_media_lag1',
                'ad_spend', 'non_social_media', 'social_media', 'competitor_ad_spend', 'retail_traffic_roll3',
                'cpi', 'unemployment', 'fuel_price', 'consumer_confidence', 'gdp_growth', 'interest_rate',
                'promo_discount', 'retail_traffic', 'holiday_indicator']
target_col = 'monthly_sales'
look_back = 3

# Result placeholders
final_res_df = pd.DataFrame()
train_fitted_list = []

# Loop through each product_id
for item_id in df['product_id'].unique():
    try:
        id_data = df[df['product_id'] == item_id].copy()
        id_data = id_data.sort_values('date')
        id_data['date'] = pd.to_datetime(id_data['date'])
        id_data.reset_index(drop=True, inplace=True)

        # Split data
        train_data = id_data[id_data['date'] < '2024-01-01'].copy()
        test_data = id_data[(id_data['date'] >= '2024-01-01') & (id_data['date'] <= '2024-12-01')].copy()

        if len(train_data) < look_back + 1 or test_data.empty:
            continue

        # Fit UCM model
        model = sm.tsa.UnobservedComponents(
            train_data[target_col],
            level='local level',
            trend=True,
            exog=train_data[feature_cols]
        )
        model_fitted = model.fit(disp=False)

        # Training accuracy
        train_fitted = model_fitted.fittedvalues
        y_train_actual = train_data[target_col].iloc[look_back:]
        y_train_pred = train_fitted.iloc[look_back:]
        mape_train = mean_absolute_percentage_error(y_train_actual, y_train_pred) * 100
        r2_train = r2_score(y_train_actual, y_train_pred)

        # Forecast for Jan–Dec 2024
        test_exog = test_data[feature_cols]
        forecast_test = model_fitted.get_forecast(steps=12, exog=test_exog)
        test_forecast_values = forecast_test.predicted_mean.tolist()
        test_actual = test_data[target_col].values
        mape_test = mean_absolute_percentage_error(test_actual, test_forecast_values) * 100
        r2_test = r2_score(test_actual, test_forecast_values)

        # Forecast for Jan–Dec 2025
        future_months = pd.date_range(start='2025-01-01', periods=12, freq='MS')
        predicted_sales = []
        extended_df = pd.concat([train_data, test_data], ignore_index=True)

        for i, pred_month in enumerate(future_months):
            # Rolling 3-month MA window: previous 3 available months
            ma_end = pred_month - pd.DateOffset(months=1)
            ma_start = ma_end - pd.DateOffset(months=2)
            ma_window = pd.date_range(start=ma_start, end=ma_end, freq='MS')

            ma_rows = []
            for win_month in ma_window:
                if win_month in extended_df['date'].values:
                    row = extended_df[extended_df['date'] == win_month].iloc[0]
                else:
                    idx = list(future_months).index(win_month)
                    predicted_val = predicted_sales[idx]
                    prev_month = future_months[idx - 1] if idx > 0 else extended_df['date'].iloc[-1]
                    prev_features = extended_df[extended_df['date'] == prev_month].iloc[0]
                    row_dict = {'date': win_month, target_col: predicted_val}
                    for col in feature_cols:
                        row_dict[col] = prev_features[col]
                    row = pd.Series(row_dict)
                ma_rows.append(row)

            ma_data = pd.DataFrame(ma_rows)
            exog_ma = ma_data[feature_cols].mean().to_frame().T
            forecast_future = model_fitted.get_forecast(steps=1, exog=exog_ma)
            next_pred = forecast_future.predicted_mean.iloc[0]
            predicted_sales.append(next_pred)

            new_row = {'date': pred_month, target_col: next_pred}
            for col in feature_cols:
                new_row[col] = exog_ma[col].values[0]
            new_row['product_id'] = item_id
            extended_df = pd.concat([extended_df, pd.DataFrame([new_row])], ignore_index=True)

        # Results DataFrame
        forecast_months = pd.date_range(start='2024-01-01', periods=24, freq='MS').strftime('%Y-%m-%d').tolist()
        actuals = test_actual.tolist() + [np.nan] * 12
        predictions = test_forecast_values + predicted_sales
        mape_values = [mape_test] * 12 + [np.nan] * 12
        r2_values = [r2_test] * 12 + [np.nan] * 12

        store_res_df = pd.DataFrame({
            'id': [item_id] * 24,
            'month': forecast_months,
            'actual': actuals,
            'predicted': predictions,
            'train_mape': [mape_train] * 24,
            'train_r2': [r2_train] * 24,
            'test_mape': mape_values,
            'test_r2': r2_values
        })

        final_res_df = pd.concat([final_res_df, store_res_df], axis=0)

        # Training fitted values
        train_months = train_data['date'].iloc[look_back:].values
        train_fitted_df = pd.DataFrame({
            'id': [item_id] * len(train_months),
            'month': train_months,
            'actual': y_train_actual.values,
            'train_fitted': y_train_pred.values,
            'train_mape': [mape_train] * len(train_months),
            'train_r2': [r2_train] * len(train_months)
        })
        train_fitted_list.append(train_fitted_df)

    except Exception as e:
        print(f"Model failed for ID {item_id}: {e}")
        continue

# Final result consolidation
final_res_df.reset_index(drop=True, inplace=True)
all_train_fitted_df = pd.concat(train_fitted_list, axis=0).reset_index(drop=True)

# Show results
print(final_res_df)
print(all_train_fitted_df)


       id       month    actual      predicted  train_mape  train_r2  \
0    P001  2024-01-01  453894.0  454756.264043    1.867699  0.889819   
1    P001  2024-02-01  410753.0  404713.430801    1.867699  0.889819   
2    P001  2024-03-01  438814.0  449818.746590    1.867699  0.889819   
3    P001  2024-04-01  444580.0  446840.895547    1.867699  0.889819   
4    P001  2024-05-01  384161.0  383630.148002    1.867699  0.889819   
..    ...         ...       ...            ...         ...       ...   
355  P015  2025-08-01       NaN  394010.924074    2.033524  0.889045   
356  P015  2025-09-01       NaN  393799.175023    2.033524  0.889045   
357  P015  2025-10-01       NaN  394024.269897    2.033524  0.889045   
358  P015  2025-11-01       NaN  393944.789665    2.033524  0.889045   
359  P015  2025-12-01       NaN  393922.744862    2.033524  0.889045   

     test_mape   test_r2  
0     1.701756  0.822171  
1     1.701756  0.822171  
2     1.701756  0.822171  
3     1.701756  0.822171  


In [14]:
# final_res_df.to_excel('ucm_final_12_forecasting_MA.xlsx',index=False)
# all_train_fitted_df.to_excel('ucm_final_12_MA.xlsx',index=False)

## xgboost

In [15]:
# 2 months

In [16]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from xgboost import XGBRegressor

# Feature and target columns
feature_cols = ['product_price', 'inventory_level', 'ad_spend_lag1', 'social_media_roll3', 'social_media_lag1',
                'ad_spend', 'non_social_media', 'social_media', 'competitor_ad_spend', 'retail_traffic_roll3',
                'cpi', 'unemployment', 'fuel_price', 'consumer_confidence', 'gdp_growth', 'interest_rate',
                'promo_discount', 'retail_traffic', 'holiday_indicator']
target_col = 'monthly_sales'

# Helper function
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

look_back = 3
final_res_df = pd.DataFrame()
train_fitted_list = []

df['date'] = pd.to_datetime(df['date'])  # Convert once

for item_id in df['product_id'].unique():
    print(f"Processing: {item_id}")
    id_data = df[df['product_id'] == item_id].copy()
    id_data = id_data.sort_values('date')

    # Train: up to Oct 2024
    train_data = id_data[id_data['date'] < pd.to_datetime('2024-11-01')]

    # Test: Nov and Dec 2024
    test_data = id_data[(id_data['date'] >= '2024-11-01') & (id_data['date'] <= pd.to_datetime('2024-12-31'))]

    # Forecast: Jan and Feb 2025
    future_months = pd.to_datetime(['2025-01-01', '2025-02-01'])

    if len(train_data) < look_back + 1 or test_data.empty:
        continue

    try:
        model = XGBRegressor()
        model.fit(train_data[feature_cols], train_data[target_col])

        # Train metrics
        y_train_pred = model.predict(train_data[feature_cols])
        y_train_actual = train_data[target_col]
        mape_train = calculate_mape(y_train_actual, y_train_pred)
        r2_train = r2_score(y_train_actual, y_train_pred)

        # Test metrics
        y_test_pred = model.predict(test_data[feature_cols])
        y_test_actual = test_data[target_col]
        mape_test = calculate_mape(y_test_actual, y_test_pred)
        r2_test = r2_score(y_test_actual, y_test_pred)

        # Forecasting for Jan & Feb 2025 using rolling 2-month window
        predict_values = []
        for i, pred_month in enumerate(future_months):
            if i == 0:
                ma_window = pd.to_datetime(['2024-11-01', '2024-12-01'])
            elif i == 1:
                ma_window = pd.to_datetime(['2024-12-01', '2025-01-01'])

            ma_window_features = id_data[id_data['date'].isin(ma_window)][feature_cols]

            if ma_window_features.empty:
                feature_row = train_data[feature_cols].iloc[[-1]]
            else:
                feature_row = ma_window_features.mean().to_frame().T

            pred_value = model.predict(feature_row)[0]
            predict_values.append(pred_value)

        # Create result DataFrame
        forecast_months = list(test_data['date'].dt.strftime('%Y-%m-%d')) + [d.strftime('%Y-%m-%d') for d in future_months]
        actuals = y_test_actual.tolist() + [np.nan] * len(future_months)
        predictions = y_test_pred.tolist() + predict_values
        mape_values = [mape_test] * len(test_data) + [np.nan] * len(future_months)
        r2_values = [r2_test] * len(test_data) + [np.nan] * len(future_months)

        store_res_df = pd.DataFrame({
            'id': [item_id] * len(forecast_months),
            'month': forecast_months,
            'actual': actuals,
            'predicted': predictions,
            'train_r2': [r2_train] * len(forecast_months),
            'test_mape': mape_values,
            'test_r2': r2_values
        })
        final_res_df = pd.concat([final_res_df, store_res_df])

        train_fitted_df = pd.DataFrame({
            'id': [item_id] * len(train_data),
            'month': train_data['date'].dt.strftime('%Y-%m-%d'),
            'actual': y_train_actual,
            'train_fitted': y_train_pred,
            'train_mape': [mape_train] * len(train_data),
            'train_r2': [r2_train] * len(train_data)
        })
        train_fitted_list.append(train_fitted_df)

    except Exception as e:
        print(f"XGBoost failed for {item_id}: {e}")

final_res_df.reset_index(drop=True, inplace=True)
all_train_fitted_df = pd.concat(train_fitted_list, axis=0).reset_index(drop=True)

print(final_res_df)
print(all_train_fitted_df)


Processing: P001
Processing: P002
Processing: P003
Processing: P004
Processing: P005
Processing: P006
Processing: P007
Processing: P008
Processing: P009
Processing: P010
Processing: P011
Processing: P012
Processing: P013
Processing: P014
Processing: P015
      id       month     actual     predicted  train_r2  test_mape     test_r2
0   P001  2024-11-01   447383.0  4.470908e+05       1.0   0.600784   -5.073514
1   P001  2024-12-01   444480.0  4.495304e+05       1.0   0.600784   -5.073514
2   P001  2025-01-01        NaN  4.447896e+05       1.0        NaN         NaN
3   P001  2025-02-01        NaN  4.495304e+05       1.0        NaN         NaN
4   P002  2024-11-01   925066.0  9.017719e+05       1.0   1.721591  -33.367157
5   P002  2024-12-01   919084.0  9.105817e+05       1.0   1.721591  -33.367157
6   P002  2025-01-01        NaN  8.895899e+05       1.0        NaN         NaN
7   P002  2025-02-01        NaN  9.105817e+05       1.0        NaN         NaN
8   P003  2024-11-01   813813.0  8

In [17]:
# final_res_df.to_excel('xgboost_final_2_forecasting_MA.xlsx',index=False)
# all_train_fitted_df.to_excel('xgboost_final_2_MA.xlsx',index=False)

In [18]:
# 3 months

In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from xgboost import XGBRegressor

# Feature and target columns
feature_cols = ['product_price', 'inventory_level', 'ad_spend_lag1', 'social_media_roll3', 'social_media_lag1',
                'ad_spend', 'non_social_media', 'social_media', 'competitor_ad_spend', 'retail_traffic_roll3',
                'cpi', 'unemployment', 'fuel_price', 'consumer_confidence', 'gdp_growth', 'interest_rate',
                'promo_discount', 'retail_traffic', 'holiday_indicator']
target_col = 'monthly_sales'

# Helper function
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

look_back = 3
final_res_df = pd.DataFrame()
train_fitted_list = []

df['date'] = pd.to_datetime(df['date'])  # Convert once

for item_id in df['product_id'].unique():
    print(f"Processing: {item_id}")
    id_data = df[df['product_id'] == item_id].copy()
    id_data = id_data.sort_values('date')

    # Train up to Sep 2024
    train_data = id_data[id_data['date'] <= pd.to_datetime('2024-09-01')]

    # Test: Oct, Nov, Dec 2024
    test_data = id_data[(id_data['date'] >= '2024-10-01') & (id_data['date'] <= '2024-12-01')]

    # Forecast: Jan, Feb, Mar 2025
    future_months = pd.to_datetime(['2025-01-01', '2025-02-01', '2025-03-01'])

    if len(train_data) < look_back + 1 or test_data.empty:
        continue

    try:
        model = XGBRegressor()
        model.fit(train_data[feature_cols], train_data[target_col])

        # Train metrics
        y_train_pred = model.predict(train_data[feature_cols])
        y_train_actual = train_data[target_col]
        mape_train = calculate_mape(y_train_actual, y_train_pred)
        r2_train = r2_score(y_train_actual, y_train_pred)

        # Test metrics
        y_test_pred = model.predict(test_data[feature_cols])
        y_test_actual = test_data[target_col]
        mape_test = calculate_mape(y_test_actual, y_test_pred)
        r2_test = r2_score(y_test_actual, y_test_pred)

        # Forecasting for Jan-Mar 2025 using rolling 3-month window
        predict_values = []
        for i, pred_month in enumerate(future_months):
            if i == 0:
                ma_window = pd.to_datetime(['2024-10-01', '2024-11-01', '2024-12-01'])
            elif i == 1:
                ma_window = pd.to_datetime(['2024-11-01', '2024-12-01', '2025-01-01'])
            else:
                ma_window = pd.to_datetime(['2024-12-01', '2025-01-01', '2025-02-01'])

            ma_window_features = id_data[id_data['date'].isin(ma_window)][feature_cols]

            if ma_window_features.empty:
                feature_row = train_data[feature_cols].iloc[[-1]]
            else:
                feature_row = ma_window_features.mean().to_frame().T

            pred_value = model.predict(feature_row)[0]
            predict_values.append(pred_value)

        # Create result DataFrame
        forecast_months = list(test_data['date'].dt.strftime('%Y-%m-%d')) + [d.strftime('%Y-%m-%d') for d in future_months]
        actuals = y_test_actual.tolist() + [np.nan] * len(future_months)
        predictions = y_test_pred.tolist() + predict_values
        mape_values = [mape_test] * len(test_data) + [np.nan] * len(future_months)
        r2_values = [r2_test] * len(test_data) + [np.nan] * len(future_months)

        store_res_df = pd.DataFrame({
            'id': [item_id] * len(forecast_months),
            'month': forecast_months,
            'actual': actuals,
            'predicted': predictions,
            'train_r2': [r2_train] * len(forecast_months),
            'test_mape': mape_values,
            'test_r2': r2_values
        })
        final_res_df = pd.concat([final_res_df, store_res_df])

        train_fitted_df = pd.DataFrame({
            'id': [item_id] * len(train_data),
            'month': train_data['date'].dt.strftime('%Y-%m-%d'),
            'actual': y_train_actual,
            'train_fitted': y_train_pred,
            'train_mape': [mape_train] * len(train_data),
            'train_r2': [r2_train] * len(train_data)
        })
        train_fitted_list.append(train_fitted_df)

    except Exception as e:
        print(f"XGBoost failed for {item_id}: {e}")

final_res_df.reset_index(drop=True, inplace=True)
all_train_fitted_df = pd.concat(train_fitted_list, axis=0).reset_index(drop=True)

print(final_res_df)
print(all_train_fitted_df)


Processing: P001
Processing: P002
Processing: P003
Processing: P004
Processing: P005
Processing: P006
Processing: P007
Processing: P008
Processing: P009
Processing: P010
Processing: P011
Processing: P012
Processing: P013
Processing: P014
Processing: P015
      id       month    actual     predicted  train_r2  test_mape   test_r2
0   P001  2024-10-01  423899.0  421821.15625       1.0   0.778133  0.857084
1   P001  2024-11-01  447383.0  445379.25000       1.0   0.778133  0.857084
2   P001  2024-12-01  444480.0  450686.46875       1.0   0.778133  0.857084
3   P001  2025-01-01       NaN  446523.87500       1.0        NaN       NaN
4   P001  2025-02-01       NaN  445358.40625       1.0        NaN       NaN
..   ...         ...       ...           ...       ...        ...       ...
85  P015  2024-11-01  428080.0  436925.90625       1.0   1.228882  0.935716
86  P015  2024-12-01  373206.0  369484.53125       1.0   1.228882  0.935716
87  P015  2025-01-01       NaN  403681.21875       1.0       

In [20]:
# final_res_df.to_excel('xgboost_final_3_forecasting_MA.xlsx',index=False)
# all_train_fitted_df.to_excel('xgboost_final_3_MA.xlsx',index=False)

In [24]:
#1 year

In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from xgboost import XGBRegressor

# Feature and target columns
feature_cols = ['product_price', 'inventory_level', 'ad_spend_lag1', 'social_media_roll3', 'social_media_lag1',
                'ad_spend', 'non_social_media', 'social_media', 'competitor_ad_spend', 'retail_traffic_roll3',
                'cpi', 'unemployment', 'fuel_price', 'consumer_confidence', 'gdp_growth', 'interest_rate',
                'promo_discount', 'retail_traffic', 'holiday_indicator']
target_col = 'monthly_sales'

# Helper function
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

look_back = 3
final_res_df = pd.DataFrame()
train_fitted_list = []

df['date'] = pd.to_datetime(df['date'])  # Convert once

for item_id in df['product_id'].unique():
    print(f"Processing: {item_id}")
    id_data = df[df['product_id'] == item_id].copy()
    id_data = id_data.sort_values('date')

    # Train: up to 2023-12-01
    train_data = id_data[id_data['date'] <= pd.to_datetime('2023-12-01')]

    # Test: Jan 2024 to Dec 2024
    test_data = id_data[(id_data['date'] >= '2024-01-01') & (id_data['date'] <= '2024-12-01')]

    # Forecast future months: Jan to Dec 2025
    future_months = pd.date_range(start='2025-01-01', end='2025-12-01', freq='MS')

    if len(train_data) < look_back + 1 or test_data.empty:
        continue

    try:
        model = XGBRegressor()
        model.fit(train_data[feature_cols], train_data[target_col])

        # Train metrics
        y_train_pred = model.predict(train_data[feature_cols])
        y_train_actual = train_data[target_col]
        mape_train = calculate_mape(y_train_actual, y_train_pred)
        r2_train = r2_score(y_train_actual, y_train_pred)

        # Test metrics
        y_test_pred = model.predict(test_data[feature_cols])
        y_test_actual = test_data[target_col]
        mape_test = calculate_mape(y_test_actual, y_test_pred)
        r2_test = r2_score(y_test_actual, y_test_pred)

        # Forecasting for 2025 using rolling 12-month feature window
        predict_values = []
        for i, pred_month in enumerate(future_months):
            start_month = pd.to_datetime('2024-01-01') + pd.DateOffset(months=i)
            end_month = pd.to_datetime('2024-12-01') + pd.DateOffset(months=i)

            ma_window = pd.date_range(start=start_month, end=end_month, freq='MS')
            ma_window_features = id_data[id_data['date'].isin(ma_window)][feature_cols]

            # If no data available for that window, fallback to last available features
            if ma_window_features.empty:
                feature_row = train_data[feature_cols].iloc[[-1]]
            else:
                feature_row = ma_window_features.mean().to_frame().T  # average over rolling window

            pred_value = model.predict(feature_row)[0]
            predict_values.append(pred_value)

        forecast_months = list(test_data['date'].dt.strftime('%Y-%m-%d')) + [d.strftime('%Y-%m-%d') for d in future_months]
        actuals = y_test_actual.tolist() + [np.nan] * len(future_months)
        predictions = y_test_pred.tolist() + predict_values
        mape_values = [mape_test] * len(test_data) + [np.nan] * len(future_months)
        r2_values = [r2_test] * len(test_data) + [np.nan] * len(future_months)

        store_res_df = pd.DataFrame({
            'id': [item_id] * len(forecast_months),
            'month': forecast_months,
            'actual': actuals,
            'predicted': predictions,
            'train_r2': [r2_train] * len(forecast_months),
            'test_mape': mape_values,
            'test_r2': r2_values
        })
        final_res_df = pd.concat([final_res_df, store_res_df])

        train_fitted_df = pd.DataFrame({
            'id': [item_id] * len(train_data),
            'month': train_data['date'].dt.strftime('%Y-%m-%d'),
            'actual': y_train_actual,
            'train_fitted': y_train_pred,
            'train_mape': [mape_train] * len(train_data),
            'train_r2': [r2_train] * len(train_data)
        })
        train_fitted_list.append(train_fitted_df)

    except Exception as e:
        print(f"XGBoost failed for {item_id}: {e}")

final_res_df.reset_index(drop=True, inplace=True)
all_train_fitted_df = pd.concat(train_fitted_list, axis=0).reset_index(drop=True)

print(final_res_df)
print(all_train_fitted_df)


Processing: P001
Processing: P002
Processing: P003
Processing: P004
Processing: P005
Processing: P006
Processing: P007
Processing: P008
Processing: P009
Processing: P010
Processing: P011
Processing: P012
Processing: P013
Processing: P014
Processing: P015
       id       month    actual     predicted  train_r2  test_mape   test_r2
0    P001  2024-01-01  453894.0  446312.25000       1.0   0.609912  0.974265
1    P001  2024-02-01  410753.0  411547.28125       1.0   0.609912  0.974265
2    P001  2024-03-01  438814.0  440640.68750       1.0   0.609912  0.974265
3    P001  2024-04-01  444580.0  446048.31250       1.0   0.609912  0.974265
4    P001  2024-05-01  384161.0  389886.78125       1.0   0.609912  0.974265
..    ...         ...       ...           ...       ...        ...       ...
355  P015  2025-08-01       NaN  412651.50000       1.0        NaN       NaN
356  P015  2025-09-01       NaN  409588.28125       1.0        NaN       NaN
357  P015  2025-10-01       NaN  409826.68750       

In [23]:
# final_res_df.to_excel('xgboost_final_12_forecasting_MA.xlsx',index=False)
# all_train_fitted_df.to_excel('xgboost_final_12_MA.xlsx',index=False)

## Prophet

In [34]:
# 2 months

In [35]:
from prophet import Prophet
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error, r2_score

final_res_df = pd.DataFrame()
train_fitted_list = []

for item_id in df['product_id'].unique():
    print(f"Processing: {item_id}")
    id_data = df[df['product_id'] == item_id].copy()
    id_data = id_data.sort_values('date')
    
    # Rename for Prophet
    id_data.rename(columns={'date': 'ds', 'monthly_sales': 'y'}, inplace=True)
    
    # Train/Test Split
    train_data = id_data[id_data['ds'] < '2024-11-01'].copy()
    test_data = id_data[(id_data['ds'] >= '2024-11-01') & (id_data['ds'] <= '2024-12-01')].copy()
    
    future_months = pd.to_datetime(['2025-01-01', '2025-02-01'])
    forecast_months = pd.to_datetime(['2024-11-01', '2024-12-01', '2025-01-01', '2025-02-01'])
    
    if len(train_data) < 6 or test_data.empty:
        continue
    
    try:
        # Train Prophet model
        model = Prophet()
        model.fit(train_data)

        # --- Train Fitted Values ---
        train_pred = model.predict(train_data[['ds']])
        train_fitted = train_pred['yhat'].values
        y_train_actual = train_data['y'].values

        mape_train = mean_absolute_percentage_error(y_train_actual, train_fitted)
        train_r2 = r2_score(y_train_actual, train_fitted)

        # Save train fitted results
        train_months = train_data['ds'].values
        train_fitted_df = pd.DataFrame({
            'id': [item_id] * len(train_months),
            'month': train_months,
            'actual': y_train_actual.flatten(),
            'train_fitted': train_fitted.flatten(),
            'train_mape': [mape_train] * len(train_months),
            'train_accuracy': [100 - mape_train] * len(train_months),
            'train_r2': [train_r2] * len(train_months)
        })
        train_fitted_list.append(train_fitted_df)

        # --- Test Prediction ---
        test_pred_df = model.predict(test_data[['ds']])
        test_pred = test_pred_df['yhat'].values
        y_test_actual = test_data['y'].values

        test_mape = mean_absolute_percentage_error(y_test_actual, test_pred)
        test_r2 = r2_score(y_test_actual, test_pred)

        test_mape_filled = [np.nan] * 2 + [test_mape] * 2
        test_acc_list = [np.nan] * 2 + [100 - test_mape] * 2
        test_r2_list = [np.nan] * 2 + [test_r2] * 2
        train_mape_list = [mape_train] * 4

        # --- Moving Average Forecast for Jan–Feb 2025 ---
        moving_avg_preds = []
        for i, pred_month in enumerate(future_months):
            if i == 0:
                ma_window = pd.to_datetime(['2024-11-01', '2024-12-01'])
            elif i == 1:
                ma_window = pd.to_datetime(['2024-12-01', '2025-01-01'])

            available_vals = id_data[id_data['ds'].isin(ma_window)]['y']
            if available_vals.empty:
                avg_val = train_data['y'].iloc[-1]
            else:
                avg_val = available_vals.mean()
            moving_avg_preds.append(avg_val)

        # --- Final Assembly ---
        all_preds = np.concatenate([test_pred, moving_avg_preds])
        all_actuals = np.concatenate([y_test_actual, [np.nan] * 2])
        
        store_res_df = pd.DataFrame({
            'id': [item_id] * 4,
            'month': forecast_months,
            'actual': all_actuals,
            'predicted': all_preds,
            'train_mape': train_mape_list,
            'test_mape': test_mape_filled,
            'train_accuracy': [100 - mape_train] * 4,
            'test_accuracy': test_acc_list,
            'train_r2': [train_r2] * 4,
            'test_r2': test_r2_list
        })

        final_res_df = pd.concat([final_res_df, store_res_df], axis=0)

    except Exception as e:
        print(f"Prophet failed for {item_id}: {e}")

# Combine all train fitted
all_train_fitted_df = pd.concat(train_fitted_list, axis=0).reset_index(drop=True)
final_res_df.reset_index(drop=True, inplace=True)


Processing: P001


15:51:49 - cmdstanpy - INFO - Chain [1] start processing
15:51:49 - cmdstanpy - INFO - Chain [1] done processing
15:51:50 - cmdstanpy - INFO - Chain [1] start processing
15:51:50 - cmdstanpy - INFO - Chain [1] done processing


Processing: P002


15:51:50 - cmdstanpy - INFO - Chain [1] start processing
15:51:50 - cmdstanpy - INFO - Chain [1] done processing


Processing: P003


...

Processing: P013


15:51:54 - cmdstanpy - INFO - Chain [1] start processing
15:51:54 - cmdstanpy - INFO - Chain [1] done processing


Processing: P014


15:51:54 - cmdstanpy - INFO - Chain [1] start processing
15:51:54 - cmdstanpy - INFO - Chain [1] done processing


Processing: P015


In [36]:
# final_res_df.to_excel('prophet_final_2_forecasting_MA.xlsx',index=False)
# all_train_fitted_df.to_excel('prophet_final_2_MA.xlsx',index=False)

In [37]:
# 3 month

In [38]:
from prophet import Prophet
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error, r2_score

final_res_df = pd.DataFrame()
train_fitted_list = []

for item_id in df['product_id'].unique():
    print(f"Processing: {item_id}")
    id_data = df[df['product_id'] == item_id].copy()
    id_data = id_data.sort_values('date')
    
    # Rename for Prophet
    id_data.rename(columns={'date': 'ds', 'monthly_sales': 'y'}, inplace=True)
    
    # Train/Test Split
    train_data = id_data[id_data['ds'] <= '2024-09-01'].copy()
    test_data = id_data[(id_data['ds'] >= '2024-10-01') & (id_data['ds'] <= '2024-12-01')].copy()
    
    future_months = pd.to_datetime(['2025-01-01', '2025-02-01', '2025-03-01'])
    forecast_months = pd.to_datetime(['2024-10-01', '2024-11-01', '2024-12-01',
                                      '2025-01-01', '2025-02-01', '2025-03-01'])
    
    if len(train_data) < 6 or test_data.empty:
        continue
    
    try:
        # Train Prophet model
        model = Prophet()
        model.fit(train_data)

        # --- Train Fitted Values ---
        train_pred = model.predict(train_data[['ds']])
        train_fitted = train_pred['yhat'].values
        y_train_actual = train_data['y'].values

        mape_train = mean_absolute_percentage_error(y_train_actual, train_fitted)
        train_r2 = r2_score(y_train_actual, train_fitted)

        # Save train fitted results
        train_months = train_data['ds'].values
        train_fitted_df = pd.DataFrame({
            'id': [item_id] * len(train_months),
            'month': train_months,
            'actual': y_train_actual.flatten(),
            'train_fitted': train_fitted.flatten(),
            'train_mape': [mape_train] * len(train_months),
            'train_accuracy': [100 - mape_train] * len(train_months),
            'train_r2': [train_r2] * len(train_months)
        })
        train_fitted_list.append(train_fitted_df)

        # --- Test Prediction ---
        test_pred_df = model.predict(test_data[['ds']])
        test_pred = test_pred_df['yhat'].values
        y_test_actual = test_data['y'].values

        test_mape = mean_absolute_percentage_error(y_test_actual, test_pred)
        test_r2 = r2_score(y_test_actual, test_pred)

        test_mape_filled = [np.nan] * 3 + [test_mape] * 3
        test_acc_list = [np.nan] * 3 + [100 - test_mape] * 3
        test_r2_list = [np.nan] * 3 + [test_r2] * 3
        train_mape_list = [mape_train] * 6

        # --- Moving Average Forecast for Jan–Mar 2025 ---
        moving_avg_preds = []
        for i, pred_month in enumerate(future_months):
            if i == 0:
                ma_window = pd.to_datetime(['2024-10-01', '2024-11-01', '2024-12-01'])
            elif i == 1:
                ma_window = pd.to_datetime(['2024-11-01', '2024-12-01', '2025-01-01'])
            else:
                ma_window = pd.to_datetime(['2024-12-01', '2025-01-01', '2025-02-01'])

            available_vals = id_data[id_data['ds'].isin(ma_window)]['y']
            if available_vals.empty:
                avg_val = train_data['y'].iloc[-1]
            else:
                avg_val = available_vals.mean()
            moving_avg_preds.append(avg_val)

        # --- Final Assembly ---
        all_preds = np.concatenate([test_pred, moving_avg_preds])
        all_actuals = np.concatenate([y_test_actual, [np.nan] * 3])
        
        store_res_df = pd.DataFrame({
            'id': [item_id] * 6,
            'month': forecast_months,
            'actual': all_actuals,
            'predicted': all_preds,
            'train_mape': train_mape_list,
            'test_mape': test_mape_filled,
            'train_accuracy': [100 - mape_train] * 6,
            'test_accuracy': test_acc_list,
            'train_r2': [train_r2] * 6,
            'test_r2': test_r2_list
        })

        final_res_df = pd.concat([final_res_df, store_res_df], axis=0)

    except Exception as e:
        print(f"Prophet failed for {item_id}: {e}")

# Combine all train fitted
all_train_fitted_df = pd.concat(train_fitted_list, axis=0).reset_index(drop=True)
final_res_df.reset_index(drop=True, inplace=True)


15:51:54 - cmdstanpy - INFO - Chain [1] start processing
15:51:54 - cmdstanpy - INFO - Chain [1] done processing


Processing: P001


15:51:55 - cmdstanpy - INFO - Chain [1] start processing


Processing: P002


15:51:55 - cmdstanpy - INFO - Chain [1] done processing
15:51:55 - cmdstanpy - INFO - Chain [1] start processing
15:51:55 - cmdstanpy - INFO - Chain [1] done processing


...

Processing: P013


15:51:58 - cmdstanpy - INFO - Chain [1] start processing
15:51:59 - cmdstanpy - INFO - Chain [1] done processing


Processing: P014


15:51:59 - cmdstanpy - INFO - Chain [1] start processing
15:51:59 - cmdstanpy - INFO - Chain [1] done processing


Processing: P015


In [39]:
# final_res_df.to_excel('prophet_final_3_forecasting_MA.xlsx',index=False)
# all_train_fitted_df.to_excel('prophet_final_3_MA.xlsx',index=False)

In [40]:
# 1 year

In [41]:
from prophet import Prophet
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error, r2_score

final_res_df = pd.DataFrame()
train_fitted_list = []

# Forecast horizon
future_months = pd.date_range(start='2025-01-01', end='2025-12-01', freq='MS')
forecast_months = pd.date_range(start='2024-01-01', end='2025-12-01', freq='MS')

for item_id in df['product_id'].unique():
    print(f"Processing: {item_id}")
    id_data = df[df['product_id'] == item_id].copy()
    id_data = id_data.sort_values('date')

    # Rename for Prophet
    id_data.rename(columns={'date': 'ds', 'monthly_sales': 'y'}, inplace=True)

    # Train/Test Split
    train_data = id_data[id_data['ds'] < '2024-01-01'].copy()
    test_data = id_data[(id_data['ds'] >= '2024-01-01') & (id_data['ds'] <= '2024-12-01')].copy()

    if len(train_data) < 12 or test_data.empty:
        continue

    try:
        # Train Prophet model
        model = Prophet()
        model.fit(train_data)

        # Train Fitted
        train_pred = model.predict(train_data[['ds']])
        train_fitted = train_pred['yhat'].values
        y_train_actual = train_data['y'].values

        mape_train = mean_absolute_percentage_error(y_train_actual, train_fitted)
        train_r2 = r2_score(y_train_actual, train_fitted)

        train_fitted_df = pd.DataFrame({
            'id': [item_id] * len(train_data),
            'month': train_data['ds'].values,
            'actual': y_train_actual.flatten(),
            'train_fitted': train_fitted.flatten(),
            'train_mape': [mape_train] * len(train_data),
            'train_accuracy': [100 - mape_train] * len(train_data),
            'train_r2': [train_r2] * len(train_data)
        })
        train_fitted_list.append(train_fitted_df)

        # Test Prediction
        test_pred_df = model.predict(test_data[['ds']])
        test_pred = test_pred_df['yhat'].values
        y_test_actual = test_data['y'].values

        test_mape = mean_absolute_percentage_error(y_test_actual, test_pred)
        test_r2 = r2_score(y_test_actual, test_pred)

        # Moving Average Forecast (12 months)
        moving_avg_preds = []
        for i, pred_month in enumerate(future_months):
            start_window = pred_month - pd.DateOffset(months=12)
            end_window = pred_month - pd.DateOffset(months=1)

            ma_window = id_data[(id_data['ds'] >= start_window) & (id_data['ds'] <= end_window)]['y']
            if ma_window.empty:
                avg_val = train_data['y'].iloc[-1]
            else:
                avg_val = ma_window.mean()
            moving_avg_preds.append(avg_val)

        all_preds = np.concatenate([test_pred, moving_avg_preds])
        all_actuals = np.concatenate([y_test_actual, [np.nan] * 12])

        # Build result dataframe
        total_len = len(forecast_months)
        store_res_df = pd.DataFrame({
            'id': [item_id] * total_len,
            'month': forecast_months,
            'actual': all_actuals,
            'predicted': all_preds,
            'train_mape': [mape_train] * total_len,
            'test_mape': [np.nan] * 12 + [test_mape] * 12,
            'train_accuracy': [100 - mape_train] * total_len,
            'test_accuracy': [np.nan] * 12 + [100 - test_mape] * 12,
            'train_r2': [train_r2] * total_len,
            'test_r2': [np.nan] * 12 + [test_r2] * 12
        })

        final_res_df = pd.concat([final_res_df, store_res_df], axis=0)

    except Exception as e:
        print(f"Prophet failed for {item_id}: {e}")

# Final outputs
all_train_fitted_df = pd.concat(train_fitted_list, axis=0).reset_index(drop=True)
final_res_df.reset_index(drop=True, inplace=True)


15:51:59 - cmdstanpy - INFO - Chain [1] start processing
15:51:59 - cmdstanpy - INFO - Chain [1] done processing


Processing: P001


15:51:59 - cmdstanpy - INFO - Chain [1] start processing
15:52:00 - cmdstanpy - INFO - Chain [1] done processing


Processing: P002


15:52:00 - cmdstanpy - INFO - Chain [1] start processing
15:52:00 - cmdstanpy - INFO - Chain [1] done processing


...

Processing: P013


15:52:04 - cmdstanpy - INFO - Chain [1] start processing
15:52:04 - cmdstanpy - INFO - Chain [1] done processing


Processing: P014


15:52:04 - cmdstanpy - INFO - Chain [1] start processing
15:52:04 - cmdstanpy - INFO - Chain [1] done processing


Processing: P015


In [42]:
# final_res_df.to_excel('prophet_final_12_forecasting_MA.xlsx',index=False)
# all_train_fitted_df.to_excel('prophet_final_12_MA.xlsx',index=False)

## lstm

In [25]:
#2 months

In [26]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

def create_multivariate_dataset(X, y, look_back):
    Xs, ys = [], []
    for i in range(len(X) - look_back):
        Xs.append(X[i:i + look_back])
        ys.append(y[i + look_back])
    return np.array(Xs), np.array(ys)

# Config
feature_cols = ['product_price', 'inventory_level', 'ad_spend_lag1', 'social_media_roll3',
                'social_media_lag1', 'ad_spend', 'non_social_media', 'social_media',
                'competitor_ad_spend', 'retail_traffic_roll3', 'cpi', 'unemployment',
                'fuel_price', 'consumer_confidence', 'gdp_growth', 'interest_rate',
                'promo_discount', 'retail_traffic', 'holiday_indicator']

target_col = 'monthly_sales'
look_back = 5

# Get product IDs
ids = df['product_id'].unique()
final_res_df = pd.DataFrame()
train_fitted_list = []

for id in ids:
    data = df[df['product_id'] == id].sort_values('date')

    # Train data up to Oct 2024
    train_raw = data[data['date'] < '2024-11-01']
    
    # Test for Nov–Dec 2024
    test_period = ['2024-11-01', '2024-12-01']
    test_raw = data[data['date'].isin(test_period)]
    
    # Features and Target
    X_train_raw = train_raw[feature_cols].copy()
    y_train_raw = train_raw[target_col].copy()

    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_scaled = scaler_x.fit_transform(X_train_raw)
    y_scaled = scaler_y.fit_transform(y_train_raw.values.reshape(-1, 1))

    X_train, y_train = create_multivariate_dataset(X_scaled, y_scaled, look_back)

    # LSTM Model
    model = Sequential()
    model.add(LSTM(64, input_shape=(look_back, len(feature_cols)), return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')

    early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=100, batch_size=8, verbose=0, callbacks=[early_stop])

    # Evaluate on Train
    train_fitted = model.predict(X_train)
    y_train_actual = scaler_y.inverse_transform(y_train.reshape(-1, 1))
    train_fitted_inv = scaler_y.inverse_transform(train_fitted)
    mape_train = mean_absolute_percentage_error(y_train_actual, train_fitted_inv) * 100
    train_r2 = r2_score(y_train_actual, train_fitted_inv)

    # Save train fitted results
    train_months = train_raw['date'].iloc[look_back:].values
    train_fitted_df = pd.DataFrame({
        'id': [id] * len(train_months),
        'month': train_months,
        'actual': y_train_actual.flatten(),
        'train_fitted': train_fitted_inv.flatten(),
        'train_mape': [mape_train] * len(train_months),
        'train_accuracy': [100 - mape_train] * len(train_months),
        'train_r2': [train_r2] * len(train_months)
    })

    train_fitted_list.append(train_fitted_df)

    # Test prediction (Nov, Dec)
    X_test_raw = test_raw[feature_cols]
    y_test = test_raw[target_col].values.reshape(-1, 1)
    X_test_scaled = scaler_x.transform(X_test_raw)

    test_input = list(X_scaled[-look_back:].copy())
    predictions = []

    for i in range(2):  # Predict Nov, Dec
        input_arr = np.array(test_input[-look_back:]).reshape(1, look_back, len(feature_cols))
        pred = model.predict(input_arr, verbose=0)
        predictions.append(pred[0])
        test_input.append(X_test_scaled[i])

    pred_inv = scaler_y.inverse_transform(predictions).flatten()
    y_test_inv = y_test.flatten()

    mape_test = mean_absolute_percentage_error(y_test_inv, pred_inv) * 100
    r2_test = r2_score(y_test_inv, pred_inv)

    result_df = pd.DataFrame({
        'id': [id] * 2,
        'month': test_period,
        'actual': y_test_inv,
        'predicted': pred_inv,
        'mape_test': [mape_test] * 2,
        'test_accuracy': [100 - mape_test] * 2,
        'test_r2': [r2_test] * 2
    })

    # Moving average forecast for Jan–feb 2025
    last_2_months = data[data['date'].isin(test_period)].copy()
    # last_features = last_2_months[feature_cols].mean()
    ma_forecasts = []

    # print("\n--- Moving Average Forecasts for Jan–feb 2025 ---")
    # for i, month in enumerate(['2025-01-01', '2025-02-01']):
    #     input_ma = scaler_x.transform(last_features.values.reshape(1, -1))
    #     input_seq = test_input[-look_back:]
    #     input_seq.append(input_ma[0])
    #     input_arr = np.array(input_seq[-look_back:]).reshape(1, look_back, len(feature_cols))
    #     pred = model.predict(input_arr, verbose=0)
    #     pred_inv_ma = scaler_y.inverse_transform(pred)[0][0]
    #     ma_forecasts.append(pred_inv_ma)

    #     print(f"\nMonth: {month}")
    #     print("Used Moving Average Features:")
    #     print(pd.Series(last_features.values, index=feature_cols))
    #     print(f"Forecasted Sales: {pred_inv_ma:.2f}")

    #     last_features = pd.concat([last_2_months[feature_cols], pd.DataFrame([last_features])]).tail(3).mean()
    #     test_input.append(input_ma[0])

    # Initialize with test data's last 2 months
    last_features_list = last_2_months[feature_cols].copy()
    
    print("\n--- Moving Average Forecasts for Jan–Feb 2025 ---")
    for i, month in enumerate(['2025-01-01', '2025-02-01']):
        # Calculate average of the latest 2 or 3 months of features
        last_features = last_features_list.tail(2).mean()
    
        # Scale and prepare input
        input_ma = scaler_x.transform(last_features.values.reshape(1, -1))
        input_seq = test_input[-look_back:]
        input_seq.append(input_ma[0])
        input_arr = np.array(input_seq[-look_back:]).reshape(1, look_back, len(feature_cols))
    
        # Predict
        pred = model.predict(input_arr, verbose=0)
        pred_inv_ma = scaler_y.inverse_transform(pred)[0][0]
        ma_forecasts.append(pred_inv_ma)
    
        print(f"\nMonth: {month}")
        print("Used Moving Average Features:")
        print(pd.Series(last_features.values, index=feature_cols))
        print(f"Forecasted Sales: {pred_inv_ma:.2f}")
    
        # Update rolling features with predicted month
        new_feature_row = pd.DataFrame([last_features], columns=feature_cols)
        last_features_list = pd.concat([last_features_list, new_feature_row], ignore_index=True)
    
        # Update test input for next LSTM input
        test_input.append(input_ma[0])


    # Add moving avg results to result_df
    ma_result_df = pd.DataFrame({
        'id': [id] * 2,
        'month': ['2025-01-01', '2025-02-01'],
        'actual': [np.nan] * 2,
        'predicted': ma_forecasts,
        # 'mape': [np.nan] * 3,
        # 'accuracy': [np.nan] * 3,
        # 'r2': [np.nan] * 3
    })

    result_df = pd.concat([result_df, ma_result_df], axis=0)
    final_res_df = pd.concat([final_res_df, result_df], axis=0)

# Final results
all_train_fitted_df = pd.concat(train_fitted_list, axis=0).reset_index(drop=True)
final_res_df.reset_index(drop=True, inplace=True)

print("\n===== Final Result (All Products) =====")
print(final_res_df)

print("\n===== Training Fitted Results (All Products) =====")
print(all_train_fitted_df)


C:\Users\HarshaGupta\AppData\Local\Temp\ipykernel_3092\3056898286.py:40: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  test_raw = data[data['date'].isin(test_period)]


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step


C:\Users\HarshaGupta\AppData\Local\Temp\ipykernel_3092\3056898286.py:117: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  last_2_months = data[data['date'].isin(test_period)].copy()



--- Moving Average Forecasts for Jan–Feb 2025 ---

Month: 2025-01-01
Used Moving Average Features:
product_price            102.665000
inventory_level         4344.500000
ad_spend_lag1           6685.500000
social_media_roll3      1908.666667
social_media_lag1       1986.500000
ad_spend                6515.500000
non_social_media        2388.500000
social_media            1943.000000
competitor_ad_spend     7385.000000
retail_traffic_roll3    5783.833333
cpi                        2.690000
unemployment               5.105000
fuel_price                 3.075000
consumer_confidence       76.275000
gdp_growth                 2.615000
interest_rate              2.665000
promo_discount            25.660000
retail_traffic          5770.500000
holiday_indicator          1.000000
dtype: float64
Forecasted Sales: 466402.34

Month: 2025-02-01
Used Moving Average Features:
product_price            103.392500
inventory_level         4306.750000
ad_spend_lag1           6834.250000
social_media_rol

C:\Users\HarshaGupta\AppData\Local\Temp\ipykernel_3092\3056898286.py:40: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  test_raw = data[data['date'].isin(test_period)]


...


--- Moving Average Forecasts for Jan–Feb 2025 ---

Month: 2025-01-01
Used Moving Average Features:
product_price             71.360000
inventory_level         4403.000000
ad_spend_lag1           4722.500000
social_media_roll3      1982.000000
social_media_lag1       1899.500000
ad_spend                5348.500000
non_social_media        1276.000000
social_media            2091.000000
competitor_ad_spend     4373.000000
retail_traffic_roll3    5642.833333
cpi                        2.770000
unemployment               5.205000
fuel_price                 3.110000
consumer_confidence       75.140000
gdp_growth                 2.715000
interest_rate              2.535000
promo_discount            23.285000
retail_traffic          5903.000000
holiday_indicator          1.000000
dtype: float64
Forecasted Sales: 339231.41

Month: 2025-02-01
Used Moving Average Features:
product_price             72.615000
inventory_level         4388.000000
ad_spend_lag1           4838.250000
social_media_rol

C:\Users\HarshaGupta\AppData\Local\Temp\ipykernel_3092\3056898286.py:40: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  test_raw = data[data['date'].isin(test_period)]


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 214ms/step


C:\Users\HarshaGupta\AppData\Local\Temp\ipykernel_3092\3056898286.py:117: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  last_2_months = data[data['date'].isin(test_period)].copy()



--- Moving Average Forecasts for Jan–Feb 2025 ---

Month: 2025-01-01
Used Moving Average Features:
product_price            114.805000
inventory_level         3485.500000
ad_spend_lag1           6951.500000
social_media_roll3      2520.166667
social_media_lag1       2591.500000
ad_spend                6447.000000
non_social_media        1587.000000
social_media            2576.000000
competitor_ad_spend     4212.500000
retail_traffic_roll3    5632.166667
cpi                        2.725000
unemployment               5.340000
fuel_price                 3.150000
consumer_confidence       75.575000
gdp_growth                 2.370000
interest_rate              2.730000
promo_discount            20.490000
retail_traffic          5777.500000
holiday_indicator          1.000000
dtype: float64
Forecasted Sales: 426976.44

Month: 2025-02-01
Used Moving Average Features:
product_price            113.347500
inventory_level         3410.250000
ad_spend_lag1           6835.250000
social_media_rol

In [27]:
# final_res_df.to_excel('Lstm_final_2_forecasting_MA.xlsx',index=False)
# all_train_fitted_df.to_excel('Lstm_final_2_MA.xlsx',index=False)

In [28]:
#3 months

In [29]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

def create_multivariate_dataset(X, y, look_back):
    Xs, ys = [], []
    for i in range(len(X) - look_back):
        Xs.append(X[i:i + look_back])
        ys.append(y[i + look_back])
    return np.array(Xs), np.array(ys)

# Config
feature_cols = ['product_price', 'inventory_level', 'ad_spend_lag1', 'social_media_roll3',
                'social_media_lag1', 'ad_spend', 'non_social_media', 'social_media',
                'competitor_ad_spend', 'retail_traffic_roll3', 'cpi', 'unemployment',
                'fuel_price', 'consumer_confidence', 'gdp_growth', 'interest_rate',
                'promo_discount', 'retail_traffic', 'holiday_indicator']

target_col = 'monthly_sales'
look_back = 5

# Get product IDs
ids = df['product_id'].unique()
final_res_df = pd.DataFrame()
train_fitted_list = []

for id in ids:
    data = df[df['product_id'] == id].sort_values('date')

    # Train data up to Sept 2024
    train_raw = data[data['date'] < '2024-10-01']
    
    # Test for Oct–Dec 2024
    test_period = ['2024-10-01', '2024-11-01', '2024-12-01']
    test_raw = data[data['date'].isin(test_period)]
    
    # Features and Target
    X_train_raw = train_raw[feature_cols].copy()
    y_train_raw = train_raw[target_col].copy()

    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_scaled = scaler_x.fit_transform(X_train_raw)
    y_scaled = scaler_y.fit_transform(y_train_raw.values.reshape(-1, 1))

    X_train, y_train = create_multivariate_dataset(X_scaled, y_scaled, look_back)

    # LSTM Model
    model = Sequential()
    model.add(LSTM(64, input_shape=(look_back, len(feature_cols)), return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')

    early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=100, batch_size=8, verbose=0, callbacks=[early_stop])

    # Evaluate on Train
    train_fitted = model.predict(X_train)
    y_train_actual = scaler_y.inverse_transform(y_train.reshape(-1, 1))
    train_fitted_inv = scaler_y.inverse_transform(train_fitted)
    mape_train = mean_absolute_percentage_error(y_train_actual, train_fitted_inv) * 100
    train_r2 = r2_score(y_train_actual, train_fitted_inv)

    # Save train fitted results
    train_months = train_raw['date'].iloc[look_back:].values
    train_fitted_df = pd.DataFrame({
        'id': [id] * len(train_months),
        'month': train_months,
        'actual': y_train_actual.flatten(),
        'train_fitted': train_fitted_inv.flatten(),
        'train_mape': [mape_train] * len(train_months),
        'train_accuracy': [100 - mape_train] * len(train_months),
        'train_r2': [train_r2] * len(train_months)
    })

    train_fitted_list.append(train_fitted_df)

    # Test prediction (Oct, Nov, Dec)
    X_test_raw = test_raw[feature_cols]
    y_test = test_raw[target_col].values.reshape(-1, 1)
    X_test_scaled = scaler_x.transform(X_test_raw)

    test_input = list(X_scaled[-look_back:].copy())
    predictions = []

    for i in range(3):  # Predict Oct, Nov, Dec
        input_arr = np.array(test_input[-look_back:]).reshape(1, look_back, len(feature_cols))
        pred = model.predict(input_arr, verbose=0)
        predictions.append(pred[0])
        test_input.append(X_test_scaled[i])

    pred_inv = scaler_y.inverse_transform(predictions).flatten()
    y_test_inv = y_test.flatten()

    mape_test = mean_absolute_percentage_error(y_test_inv, pred_inv) * 100
    r2_test = r2_score(y_test_inv, pred_inv)

    result_df = pd.DataFrame({
        'id': [id] * 3,
        'month': test_period,
        'actual': y_test_inv,
        'predicted': pred_inv,
        'mape_test': [mape_test] * 3,
        'test_accuracy': [100 - mape_test] * 3,
        'test_r2': [r2_test] * 3
    })

    # Moving average forecast for Jan–Mar 2025
    last_3_months = data[data['date'].isin(test_period)].copy()
    last_features = last_3_months[feature_cols].mean()
    ma_forecasts = []

    print("\n--- Moving Average Forecasts for Jan–Mar 2025 ---")
    for i, month in enumerate(['2025-01-01', '2025-02-01', '2025-03-01']):
        input_ma = scaler_x.transform(last_features.values.reshape(1, -1))
        input_seq = test_input[-look_back:]
        input_seq.append(input_ma[0])
        input_arr = np.array(input_seq[-look_back:]).reshape(1, look_back, len(feature_cols))
        pred = model.predict(input_arr, verbose=0)
        pred_inv_ma = scaler_y.inverse_transform(pred)[0][0]
        ma_forecasts.append(pred_inv_ma)

        print(f"\nMonth: {month}")
        print("Used Moving Average Features:")
        print(pd.Series(last_features.values, index=feature_cols))
        print(f"Forecasted Sales: {pred_inv_ma:.2f}")

        last_features = pd.concat([last_3_months[feature_cols], pd.DataFrame([last_features])]).tail(3).mean()
        test_input.append(input_ma[0])

    # Add moving avg results to result_df
    ma_result_df = pd.DataFrame({
        'id': [id] * 3,
        'month': ['2025-01-01', '2025-02-01', '2025-03-01'],
        'actual': [np.nan] * 3,
        'predicted': ma_forecasts,
        # 'mape': [np.nan] * 3,
        # 'accuracy': [np.nan] * 3,
        # 'r2': [np.nan] * 3
    })

    result_df = pd.concat([result_df, ma_result_df], axis=0)
    final_res_df = pd.concat([final_res_df, result_df], axis=0)

# Final results
all_train_fitted_df = pd.concat(train_fitted_list, axis=0).reset_index(drop=True)
final_res_df.reset_index(drop=True, inplace=True)

print("\n===== Final Result (All Products) =====")
print(final_res_df)

print("\n===== Training Fitted Results (All Products) =====")
print(all_train_fitted_df)


C:\Users\HarshaGupta\AppData\Local\Temp\ipykernel_3092\1642680404.py:40: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  test_raw = data[data['date'].isin(test_period)]


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step


C:\Users\HarshaGupta\AppData\Local\Temp\ipykernel_3092\1642680404.py:117: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  last_3_months = data[data['date'].isin(test_period)].copy()



--- Moving Average Forecasts for Jan–Mar 2025 ---

Month: 2025-01-01
Used Moving Average Features:
product_price            100.980000
inventory_level         4344.000000
ad_spend_lag1           6042.000000
social_media_roll3      1868.777778
social_media_lag1       1817.666667
ad_spend                6473.000000
non_social_media        2352.000000
social_media            1999.666667
competitor_ad_spend     6604.333333
retail_traffic_roll3    5799.333333
cpi                        2.720000
unemployment               5.206667
fuel_price                 3.080000
consumer_confidence       76.550000
gdp_growth                 2.400000
interest_rate              2.593333
promo_discount            19.656667
retail_traffic          5824.333333
holiday_indicator          0.666667
dtype: float64
Forecasted Sales: 464594.03

Month: 2025-02-01
Used Moving Average Features:
product_price            102.103333
inventory_level         4344.333333
ad_spend_lag1           6471.000000
social_media_rol

C:\Users\HarshaGupta\AppData\Local\Temp\ipykernel_3092\1642680404.py:40: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  test_raw = data[data['date'].isin(test_period)]


...


--- Moving Average Forecasts for Jan–Mar 2025 ---

Month: 2025-01-01
Used Moving Average Features:
product_price             73.080000
inventory_level         4481.333333
ad_spend_lag1           4520.000000
social_media_roll3      1965.111111
social_media_lag1       1952.000000
ad_spend                5062.666667
non_social_media        1255.666667
social_media            2012.000000
competitor_ad_spend     4938.000000
retail_traffic_roll3    5667.333333
cpi                        2.750000
unemployment               5.180000
fuel_price                 3.113333
consumer_confidence       75.450000
gdp_growth                 2.780000
interest_rate              2.560000
promo_discount            15.713333
retail_traffic          5708.333333
holiday_indicator          0.666667
dtype: float64
Forecasted Sales: 342153.62

Month: 2025-02-01
Used Moving Average Features:
product_price             71.933333
inventory_level         4429.111111
ad_spend_lag1           4655.000000
social_media_rol

C:\Users\HarshaGupta\AppData\Local\Temp\ipykernel_3092\1642680404.py:40: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  test_raw = data[data['date'].isin(test_period)]


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step


C:\Users\HarshaGupta\AppData\Local\Temp\ipykernel_3092\1642680404.py:117: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  last_3_months = data[data['date'].isin(test_period)].copy()



--- Moving Average Forecasts for Jan–Mar 2025 ---

Month: 2025-01-01
Used Moving Average Features:
product_price            113.626667
inventory_level         3543.000000
ad_spend_lag1           6110.666667
social_media_roll3      2493.222222
social_media_lag1       2467.333333
ad_spend                6692.666667
non_social_media        1634.000000
social_media            2573.000000
competitor_ad_spend     4749.000000
retail_traffic_roll3    5612.333333
cpi                        2.730000
unemployment               5.406667
fuel_price                 3.160000
consumer_confidence       76.380000
gdp_growth                 2.446667
interest_rate              2.770000
promo_discount            16.126667
retail_traffic          5683.666667
holiday_indicator          0.666667
dtype: float64
Forecasted Sales: 419223.41

Month: 2025-02-01
Used Moving Average Features:
product_price            114.412222
inventory_level         3504.666667
ad_spend_lag1           6671.222222
social_media_rol

In [30]:
# final_res_df.to_excel('Lstm_final_3_forecasting_MA.xlsx',index=False)
# all_train_fitted_df.to_excel('Lstm_final_3_MA.xlsx',index=False)

In [31]:
# 1 year 

In [32]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

def create_multivariate_dataset(X, y, look_back):
    Xs, ys = [], []
    for i in range(len(X) - look_back):
        Xs.append(X[i:i + look_back])
        ys.append(y[i + look_back])
    return np.array(Xs), np.array(ys)

# Config
feature_cols = ['product_price', 'inventory_level', 'ad_spend_lag1', 'social_media_roll3',
                'social_media_lag1', 'ad_spend', 'non_social_media', 'social_media',
                'competitor_ad_spend', 'retail_traffic_roll3', 'cpi', 'unemployment',
                'fuel_price', 'consumer_confidence', 'gdp_growth', 'interest_rate',
                'promo_discount', 'retail_traffic', 'holiday_indicator']
target_col = 'monthly_sales'
look_back = 5

ids = df['product_id'].unique()
final_res_df = pd.DataFrame()
train_fitted_list = []

for id in ids:
    data = df[df['product_id'] == id].sort_values('date')
    train_raw = data[data['date'] < '2024-01-01']
    test_raw = data[(data['date'] >= '2024-01-01') & (data['date'] <= '2024-12-01')].copy()

    if test_raw.shape[0] < 12:
        print(f"Skipping product {id}: Not enough test data ({test_raw.shape[0]} rows)")
        continue

    # Features and Target
    X_train_raw = train_raw[feature_cols].copy()
    y_train_raw = train_raw[target_col].copy()

    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_scaled = scaler_x.fit_transform(X_train_raw)
    y_scaled = scaler_y.fit_transform(y_train_raw.values.reshape(-1, 1))

    X_train, y_train = create_multivariate_dataset(X_scaled, y_scaled, look_back)

    # LSTM Model
    model = Sequential()
    model.add(LSTM(64, input_shape=(look_back, len(feature_cols)), return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')

    early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=100, batch_size=8, verbose=0, callbacks=[early_stop])

    # Evaluate on Train
    train_fitted = model.predict(X_train)
    y_train_actual = scaler_y.inverse_transform(y_train.reshape(-1, 1))
    train_fitted_inv = scaler_y.inverse_transform(train_fitted)
    mape_train = mean_absolute_percentage_error(y_train_actual, train_fitted_inv) * 100
    train_r2 = r2_score(y_train_actual, train_fitted_inv)

    train_months = train_raw['date'].iloc[look_back:].values
    train_fitted_df = pd.DataFrame({
        'id': [id] * len(train_months),
        'month': train_months,
        'actual': y_train_actual.flatten(),
        'train_fitted': train_fitted_inv.flatten(),
        'train_mape': [mape_train] * len(train_months),
        'train_accuracy': [100 - mape_train] * len(train_months),
        'train_r2': [train_r2] * len(train_months)
    })

    train_fitted_list.append(train_fitted_df)

    # Test prediction (Jan–Dec 2024)
    X_test_raw = test_raw[feature_cols]
    y_test = test_raw[target_col].values.reshape(-1, 1)
    X_test_scaled = scaler_x.transform(X_test_raw)

    test_input = list(X_scaled[-look_back:].copy())
    predictions = []

    for i in range(12):  # Jan to Dec
        input_arr = np.array(test_input[-look_back:]).reshape(1, look_back, len(feature_cols))
        pred = model.predict(input_arr, verbose=0)
        predictions.append(pred[0])
        test_input.append(X_test_scaled[i])

    pred_inv = scaler_y.inverse_transform(predictions).flatten()
    y_test_inv = y_test.flatten()

    mape_test = mean_absolute_percentage_error(y_test_inv, pred_inv) * 100
    r2_test = r2_score(y_test_inv, pred_inv)

    result_df = pd.DataFrame({
        'id': [id] * 12,
        'month': test_raw['date'].values,
        'actual': y_test_inv,
        'predicted': pred_inv,
        'mape_test': [mape_test] * 12,
        'test_accuracy': [100 - mape_test] * 12,
        'test_r2': [r2_test] * 12
    })

    # # Moving Average Forecast for Jan–Dec 2025
    # ma_forecasts = []
    # forecast_months = pd.date_range(start='2025-01-01', end='2025-12-01', freq='MS')
    # last_features_list = test_raw[feature_cols].copy()

    # for month in forecast_months:
    #     last_features = last_features_list.tail(12).mean()
    #     input_ma = scaler_x.transform(last_features.values.reshape(1, -1))
    #     input_seq = test_input[-look_back:]
    #     input_seq.append(input_ma[0])
    #     input_arr = np.array(input_seq[-look_back:]).reshape(1, look_back, len(feature_cols))
    #     pred = model.predict(input_arr, verbose=0)
    #     pred_inv_ma = scaler_y.inverse_transform(pred)[0][0]
    #     ma_forecasts.append(pred_inv_ma)
    #     new_feature_row = pd.DataFrame([last_features], columns=feature_cols)
    #     last_features_list = pd.concat([last_features_list, new_feature_row], ignore_index=True)
    #     test_input.append(input_ma[0])
        # Moving Average Forecast for Jan–Dec 2025
    ma_forecasts = []
    forecast_months = pd.date_range(start='2025-01-01', end='2025-12-01', freq='MS')
    last_features_list = test_raw[feature_cols].copy()

    print(f"\nMoving Average Inputs for Product ID: {id}")
    for month in forecast_months:
        last_features = last_features_list.tail(12).mean()
        print(f"Month: {month.strftime('%Y-%m')}, Moving Average Features:\n{last_features}\n")

        input_ma = scaler_x.transform(last_features.values.reshape(1, -1))
        input_seq = test_input[-look_back:]
        input_seq.append(input_ma[0])
        input_arr = np.array(input_seq[-look_back:]).reshape(1, look_back, len(feature_cols))
        pred = model.predict(input_arr, verbose=0)
        pred_inv_ma = scaler_y.inverse_transform(pred)[0][0]
        ma_forecasts.append(pred_inv_ma)
        new_feature_row = pd.DataFrame([last_features], columns=feature_cols)
        last_features_list = pd.concat([last_features_list, new_feature_row], ignore_index=True)
        test_input.append(input_ma[0])


    ma_result_df = pd.DataFrame({
        'id': [id] * 12,
        'month': forecast_months,
        'actual': [np.nan] * 12,
        'predicted': ma_forecasts
    })

    result_df = pd.concat([result_df, ma_result_df], axis=0)
    final_res_df = pd.concat([final_res_df, result_df], axis=0)

# Final results
all_train_fitted_df = pd.concat(train_fitted_list, axis=0).reset_index(drop=True)
final_res_df.reset_index(drop=True, inplace=True)

print("\n===== Final Result (All Products) =====")
print(final_res_df)

print("\n===== Training Fitted Results (All Products) =====")
print(all_train_fitted_df)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step

Moving Average Inputs for Product ID: P001
Month: 2025-01, Moving Average Features:
product_price             98.316667
inventory_level         4421.750000
ad_spend_lag1           6251.916667
social_media_roll3      1905.000000
social_media_lag1       1891.250000
ad_spend                6274.833333
non_social_media        2363.000000
social_media            1909.500000
competitor_ad_spend     5472.333333
retail_traffic_roll3    5864.083333
cpi                        2.692500
unemployment               5.219167
fuel_price                 3.170000
consumer_confidence       76.149167
gdp_growth                 2.434167
interest_rate              2.448333
promo_discount             9.192500
retail_traffic          5885.000000
holiday_indicator          0.166667
dtype: float64

Month: 2025-02, Moving Average Features:
product_price             97.540556
inventory_level         4438.812500
ad_spend_lag1           6291.826389
social_media_roll3      1906

In [33]:
# final_res_df.to_excel('Lstm_final_12_forecasting_MA.xlsx',index=False)
# all_train_fitted_df.to_excel('Lstm_final_12_MA.xlsx',index=False)